# Moments #

Here we show how to find the moments of the Phat distribution. As we know, the Phat distribution is a mixture model of two Carben distributions, each weighted 50%. Thus, the PDF of the Phat distribution is simply:

$$
f(x) = \sum\limits_{i=1}^nw_if_i(x)
\\f(x) = 0.5*f_{\textit{left}}(x) + 0.5*f_{\textit{right}}(x)
$$

The Carben distribution has a piecewise density function, defined for a right-tail as:

$$
f_{\textit{right}}(x) = \left\{ \begin{array}{ll}
      \frac{1}{\gamma} f_{\mu,\sigma}(x) & \text{if } y\leq a \\
      \frac{1}{\gamma} g_{\xi,a,b}(x) & \text{if } y > a\\
\end{array} \right.
$$

And for the left

$$
f_{\textit{left}}(x) = \left\{ \begin{array}{ll}
      \frac{1}{\gamma} g_{\xi,-a,b}(-x) & \text{if } y < a \\
       \frac{1}{\gamma} f_{\mu,\sigma}(x) & \text{if } y \geq a\\
\end{array} \right.
$$

The MGF for a mixture model is simply the weighted average of MGF's of its components, shown as follows:

$$
M_x(n) = \int_{-\infty}^{+\infty}e^{nx}f_X(x)\delta x
\\ = \int_{-\infty}^{+\infty}e^{nx}\left(.5*f_{\textit{left}}(x) + .5*f_{\textit{right}}(x)\right)\delta x
$$

$$
\\ = \int_{-\infty}^{+\infty}.5*e^{nx}f_{\textit{left}}(x) + .5*e^{nx}f_{\textit{right}}(x)\delta x
\\ = .5\int_{-\infty}^{+\infty}e^{nx}f_{\textit{left}}(x)\delta x + .5\int_{-\infty}^{+\infty}e^{nx}f_{\textit{right}}(x)\delta x
\\ = .5*M_{left} + .5*M_{right}
$$

Thus, to find the moments of the Phat, we must find the MGFs of the component Carbens.

## Carben Right ##

The MGF of a piecewise distribution is simply the sum of the integrals along the bounded ranges.

$$
M_{right}(n) = \frac{1}{\gamma}\int_{-\infty}^{a}e^{nx}f_{\mu,\sigma}(x)\delta x + \frac{1}{\gamma}\int_{a}^{+\infty}e^{nx}g_{\xi,a,b}(x)\delta x
$$

The body term is a truncated normal distribution. The moments of such functions are known and available in `scipy`. We know that the generalized Pareto is always restricted the interval $(a,\infty)$, so the tail term is merely the MGF of generalized Pareto scaled by $\frac{1}{\gamma}$. Thus,

$$
\frac{1}{\gamma}M_{\mu,\sigma,u}(n) + \frac{1}{\gamma}M_{\xi,a,b}(n)
$$
where: $u$ is the upper bound of the body

## Carben Left ##

Now for the left:

$$
M_{left}(n) = \frac{1}{\gamma}\int_{-\infty}^{a}e^{nx}g_{\xi,-a,b}(-x)\delta x
 + \frac{1}{\gamma}\int_{a}^{+\infty}e^{nx}f_{\mu,\sigma}(x)\delta x
\\=\frac{1}{\gamma}\int_{-a}^{+\infty}e^{nx}g_{\xi,-a,b}(x)\delta x
 + \frac{1}{\gamma}\int_{a}^{+\infty}e^{nx}f_{\mu,\sigma}(x)\delta x
$$

Once again, the tail portion is the MGF of the generalized Pareto. And once again, the body portion is a truncated normal, this time with a lower bound, so the MGF is found as:

$$
\frac{1}{\gamma}M_{\xi,-a,b}(n) + \frac{1}{\gamma}M_{\mu,\sigma,l}(n)
$$
where: $l$ is the lower bound of the body

## Mean ##

The first moment is the first derivative of $M$, so:

$$
\mu_{\textit{right}} = \frac{d}{dx}\frac{1}{\gamma}M_{\mu,\sigma,u}(1) + \frac{d}{dx}\frac{1}{\gamma}M_{\xi,a,b}(1)
\\\mu_{\textit{left}} = \frac{d}{dx}\frac{1}{\gamma}M_{\xi,-a,b}(1) + \frac{d}{dx}\frac{1}{\gamma}M_{\mu,\sigma,l}(1)
$$

In both instances, the constant multiplicative survives the differentiations. For the left tail, in order to reflect its true location, $a$, we must invert the sign:
$$
\mu_{\textit{phat}} = .5*\mu_{\textit{right}} + .5*\mu_{\textit{left}}
\\\mu_{\textit{right}} = \frac{1}{\gamma}\text{Mean}_{\mu,\sigma,u} + \frac{1}{\gamma}\text{Mean}_{\xi,a,b}
\\\mu_{\textit{left}} = -\frac{1}{\gamma}\text{Mean}_{\xi,-a,b} + \frac{1}{\gamma}\text{Mean}_{\mu,\sigma,l}
$$

Finding mean of the Phat distribution programmatically is thus fairly trivial.

In [1]:
import numpy as np
import scipy.stats as scist

import phat as ph

In [2]:
phat = ph.Phat(.1, .2, .2, .25)

bmu = scist.truncnorm(
    -np.inf, 
    phat.right.loc, 
    *phat.right.body.args
).mean()
tmu = phat.right.tail.mean()
rmu = (bmu + tmu) / phat.right.gamma

bmu = scist.truncnorm(
    phat.left.loc,
    np.inf, 
    *phat.left.body.args
).mean()
tmu = -phat.left.tail.mean()
lmu = (bmu + tmu) / phat.left.gamma

The mean of the Phat distribution is:

In [3]:
np.mean((lmu, rmu))

0.1426574558489754

We can see the mean of the Phat in the above example is close to, but slightly higher than the mean of the Gaussian body, $\mu$. This results from the greater tail index in the right tail used in the example.

In [5]:
phat.mean()

0.1426574558489754

## Variance ##

The second moment is the second derivative of $M$, so:

$$
\text{Var}_{\textit{right}} = \frac{d^2}{dx}\frac{1}{\gamma}M_{\mu,\sigma,u}(2) + \frac{d^2}{dx}\frac{1}{\gamma}M_{\xi,a,b}(2)
\\\text{Var}_{\textit{left}} = \frac{d^2}{dx}\frac{1}{\gamma}M_{\xi,-a,b}(2) + \frac{d^2}{dx}\frac{1}{\gamma}M_{\mu,\sigma,l}(2)
\\\text{Var}_{\textit{right}} = \frac{1}{\gamma}\text{Var}_{\mu,\sigma,u} + \frac{1}{\gamma}\text{Var}_{\xi,a,b}
\\\text{Var}_{\textit{left}} = \frac{1}{\gamma}\text{Var}_{\xi,-a,b} + \frac{1}{\gamma}\text{Var}_{\mu,\sigma,l}
\\\text{Var}_{\textit{phat}} = .5*\text{Var}_{\textit{right}} + .5*\text{Var}_{\textit{left}}
$$

Note that no negative is applied to the second moment as it eliminated bystanders the square.

And so the variance of the Phat is found as:

In [6]:
bvar = scist.truncnorm(
    -np.inf, 
    phat.right.loc, 
    *phat.right.body.args
).var()
tvar = phat.right.tail.var()
rvar = (bvar + tvar) / phat.right.gamma

In [7]:
bvar = scist.truncnorm(
    phat.left.loc,
    np.inf, 
    *phat.left.body.args
).var()
tvar = phat.left.tail.var()
lvar = (bvar + tvar) / phat.left.gamma

In [8]:
lvar

0.4828404012432034

The variance for the Phat distribution is then: 

In [9]:
np.mean((lvar, rvar))

0.5732923559721449

This is avialable as `var` method of the Phat distribution.

In [10]:
phat.var()

0.5732923559721449